In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
inputs = pd.read_csv('data/traincleaned.csv')
testinputs = pd.read_csv('data/testcleaned.csv')

inputs = inputs.drop(columns=['SalePrice'])
alldata = pd.concat([inputs, testinputs])
alldata.set_index('Id', inplace=True)
alldata = alldata.fillna(0)
alldata = pd.get_dummies(alldata)
alldata = alldata.drop(columns=['Unnamed: 0'])

inputs = alldata.loc[0:1460,]
inputs = inputs.values

testinputs = alldata.loc[1461:]
testinputs = testinputs.values

C:\Users\brand\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [3]:
prices = pd.read_csv('data/traincleaned.csv')
prices = prices['SalePrice']
prices = np.array(prices).reshape(-1,1)
prices

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]], dtype=int64)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

X_train, X_test, y_train, y_test = train_test_split(inputs, prices,test_size=0.33, random_state=42)

In [6]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)

params = {"max_depth":2, "eta":0.1}
model = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)

In [7]:
model_xgb = xgb.XGBRegressor(n_estimators=350, max_depth=2, learning_rate=0.1) #the params were tuned using xgb.cv
model_xgb.fit(X_train, y_train)
model_xgb.score(X_test, y_test)

0.9262361983905583

In [8]:
model_xgb.fit(inputs, prices)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=350, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [9]:
model_xgb.score(inputs, prices)

0.9689396438087249

In [10]:
pred =model_xgb.predict(testinputs)
pred 

array([118710.43 , 164661.47 , 181707.75 , ..., 163885.83 , 116771.016,
       231680.88 ], dtype=float32)

In [12]:
submittest = pd.read_csv('data/testcleaned.csv')
submittest = submittest[['Id']]
submittest['SalePrice'] = pred
submittest.to_csv('ShetuSubmission.csv',index = False)
submittest.head()

,Id,SalePrice
0,1461,118710.429688
1,1462,164661.468750
2,1463,181707.750000
3,1464,189850.281250
4,1465,197253.015625
